Using $\rho (r)$ for density

$$
\frac{dm_r(r)}{dr}=4\pi r^2 \rho(r)
$$

and 

<!--  -->
$$
\frac{dP(r)}{dr}=
$$

where $m_r(r)$ is the mass enclosed in a radius, $\rho(r)$ is the density at some radius r, and
$P(r)$ is the pressure at radius $r$.
